In [ ]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QTabWidget, QTableWidgetItem, QTableView, QHeaderView
)
from PyQt5.QtGui import QStandardItemModel, QStandardItem
from PyQt5 import uic
from data201 import db_connection # Assuming this is your utility

In [ ]:
#Tab1 and Tab2 are in the same file
class Airpollution(QTabWidget):
    """
    The Airpollution Dialog with TabWidget (Tab 1: View, Tab 2: Modify).
    """

    def __init__(self):
        """
        Load the UI and initialize components for both tabs.
        """
        super().__init__()

        # Load the UI
        self.ui = uic.loadUi('AirpollutionTab.ui', self)

        # Get tab references by index because both are named 'tab'
        self.tab_view = self.ui.widget(0)   # Tab 1: View Data
        self.tab_modify = self.ui.widget(1)  # Tab 2: Modify

        # ---------- Tab 1 Setup (view) ----------
        self.ui.pollutant_menu.currentIndexChanged.connect(self._initialize_table)
        self.ui.button.clicked.connect(self._enter_cities_of_pollutant)
        self.pollutant_table = self.ui.pollutant_table

        # Initialize pollutant menu and table
        self.initialize_pollutant_menu()
        self._initialize_table()

        # ---------- Tab 2 Setup (Modify) ----------
        self._setup_modify_tab()

    def _setup_modify_tab(self):
        """
        Set up Tab 2 (Modify) components using index since objectName can't be unique.
        """
        self.selected_column = None  # Safe initialization

        # Radio buttons and other components (already uniquely named)
        self.radio_aqid = self.ui.aqid
        self.radio_measure = self.ui.measure
        self.radio_category = self.ui.category
        self.text_input = self.ui.text_input
        self.text_input.setPlaceholderText("Enter value to search...")
        self.search_button = self.ui.search_button
        self.column_view = self.ui.column_view  # QTableWidget already present

        # Connect radio buttons and search button
        self.radio_aqid.toggled.connect(lambda checked: self.radio_button_clicked('aq_id', checked))
        self.radio_measure.toggled.connect(lambda checked: self.radio_button_clicked('measure', checked))
        self.radio_category.toggled.connect(lambda checked: self.radio_button_clicked('data_value', checked))
        self.search_button.clicked.connect(self.search_and_show_data)

    # ---------------- Tab 1 ("view") Methods ---------------- #

    def initialize_pollutant_menu(self):
        """
        Populate pollutant menu dropdown.
        """
        conn = db_connection(config_file='joinforce.ini')
        cursor = conn.cursor()

        sql = "SELECT aq_id FROM air_quality;"
        cursor.execute(sql)
        rows = cursor.fetchall()
        cursor.close()
        conn.close()

        for row in rows:
            self.ui.pollutant_menu.addItem(str(row[0]), row[0])

    def _initialize_table(self):
        """
        Set headers and prepare empty pollutant table.
        """
        self.ui.pollutant_table.setRowCount(0)
        self.ui.pollutant_table.setColumnCount(3)
        self.ui.pollutant_table.setHorizontalHeaderLabels([' District Name ', ' Measure ', ' Air Quality '])
        self._adjust_column_widths()

    def _enter_cities_of_pollutant(self):
        """
        Populate table with pollutant data for selected pollutant.
        """
        name = self.ui.pollutant_menu.currentData()
        if name is None:
            print("No pollutant selected.")
            return

        conn = db_connection(config_file='joinforce.ini')
        cursor = conn.cursor()

        sql = """
        SELECT d.district_name, a.measure, a.data_value
        FROM district d
        JOIN air_quality a ON d.district_id = a.district_id
        WHERE a.aq_id = %s
        ORDER BY a.date DESC;
        """
        cursor.execute(sql, (name,))
        rows = cursor.fetchall()
        cursor.close()
        conn.close()

        self.ui.pollutant_table.setRowCount(0)
        for row_index, row in enumerate(rows):
            self.ui.pollutant_table.insertRow(row_index)
            for column_index, data in enumerate(row):
                self.ui.pollutant_table.setItem(row_index, column_index, QTableWidgetItem(str(data)))

        self._adjust_column_widths()

    def _adjust_column_widths(self):
        """
        Auto-adjust column widths for pollutant table.
        """
        header = self.ui.pollutant_table.horizontalHeader()
        header.setSectionResizeMode(0, QHeaderView.ResizeToContents)
        header.setSectionResizeMode(1, QHeaderView.Stretch)

    # ---------------- Tab 2 ("Modify") Methods ---------------- #

    def radio_button_clicked(self, column_name, checked):
        """
        Track selected radio button (column for search).
        """
        if checked:
            print(f"Selected column: {column_name}")
            self.selected_column = column_name

    def search_and_show_data(self):
        """
        Handle search button click, validate input, and trigger search.
        """
        if not self.selected_column:
            print("Please select a column using the radio buttons.")
            return

        search_value = self.text_input.toPlainText().strip()
        if not search_value:
            print("Please enter a value to search.")
            return

        self.search_table(self.selected_column, search_value)

    def search_table(self, column_name, value):
        """
        Perform database search based on column and value, display results.
        """
        allowed_columns = {'aq_id', 'measure', 'data_value'}
        if column_name not in allowed_columns:
            print("Invalid column name selected.")
            return

        conn = db_connection(config_file='joinforce.ini')
        cursor = conn.cursor()

        sql = f"""
        SELECT aq_id, measure, data_value, district_id
        FROM joinforce_db.air_quality_category
        WHERE {column_name} = %s;
        """
        print("Executing Search SQL:", sql, (value,))

        try:
            cursor.execute(sql, (value,))
            rows = cursor.fetchall()
            if not rows:
                print("No records found for the search.")
        except Exception as e:
            print("Search Error:", e)
            rows = []

        cursor.close()
        conn.close()


        # Display results in column_view
        column_names = ['AQ ID', 'Measure', 'Data Value', 'District ID', 'Column5', 'Column6']
        self.populate_table_view(rows, column_names)

    def populate_table_view(self, rows, column_names):
        """
        Populate column_view (QTableWidget) with fetched search results.
        """
        self.column_view.setRowCount(0)
        self.column_view.setColumnCount(len(column_names))
        self.column_view.setHorizontalHeaderLabels(column_names)

        for row_index, row in enumerate(rows):
            self.column_view.insertRow(row_index)
            for column_index, data in enumerate(row):
                self.column_view.setItem(row_index, column_index, QTableWidgetItem(str(data)))



# ---- Main entry point ----

if __name__ == '__main__':
    app = QApplication(sys.argv)
    form = Airpollution()
    form.show()  # Show the tabbed interface
    sys.exit(app.exec_())
